## Load modules

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import glob
import math
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
from PIL import Image
from geometry import extract_feature_pixels, focal_length_to_pixels, calculate_height_difference_perspective

## Set input and output folders

In [ ]:
in_folder_predictions=r'D:\Wagga\RICS\all_images_predicted'
building_point_file=r'D:\Wagga\RICS\Final_Wagga.geojson'

In [ ]:
gdf=gpd.read_file(building_point_file)
gdf.head()

## Read in segmentation predictions from OneFormer

In [ ]:
prediction_files = glob.glob(os.path.join(in_folder_predictions,'*.png'))
prediction_file=prediction_files[0]
lbl_arr = np.asarray(Image.open(prediction_file))

In [ ]:
RICS_id=os.path.basename(prediction_file).split('.')[0]
print(RICS_id)

In [ ]:
def estimate_FFH(elev_foundation_top=None, elev_foundation_bottom=None,elev_stairs_top=None, elev_stairs_bottom=None, elev_frontdoor_bottom=None,elev_garagedoor_bottom=None, elev_camera=None):
    '''
    Calculate FFH using elevations of available features
    '''
    # determine ground elevation
    elev_ground=None
    if elev_garagedoor_bottom is not None:
        elev_ground=elev_garagedoor_bottom
    elif elev_stairs_bottom is not None:
        elev_ground=elev_stairs_bottom
    elif elev_foundation_bottom is not None:
        elev_ground=elev_foundation_bottom
    if elev_ground is None:
        return None
    
    elev_floor=None
    if elev_frontdoor_bottom is not None:
        elev_floor=elev_frontdoor_bottom
    elif elev_stairs_top is not None:
        elev_floor=elev_stairs_top
    elif elev_foundation_top is not None:
        elev_floor=elev_foundation_top
    if elev_floor is None:
        return None
    if elev_floor<elev_ground:
        return None
    # calculate FFH
    FFH=elev_floor-elev_ground
    return FFH

## Extract feature bottom pixels

In [ ]:
# extract bottom pixels of foundation, stairs, front door and garage door
foundation_mask=(lbl_arr==1)
frontdoor_mask = (lbl_arr==2)
garagedoor_mask = (lbl_arr==3)
stairs_mask = (lbl_arr==4)
foundation_bottom_pixels = extract_feature_pixels(foundation_mask,extract="bottom")
stairs_bottom_pixels = extract_feature_pixels(stairs_mask, extract="bottom")
frontdoor_bottom_pixels = extract_feature_pixels(frontdoor_mask,extract="bottom")
garagedoor_bottom_pixels = extract_feature_pixels(garagedoor_mask,extract="bottom")

# extract top pixels of foundation and stairs
foundation_top_pixels = extract_feature_pixels(foundation_mask,extract="top")
stairs_top_pixels = extract_feature_pixels(stairs_mask,extract="top")

In [ ]:
# # Assume `door_mask` is a binary 2D numpy array representing the mask with the door labeled as 1.
# door_mask = (lbl_arr==1)
# # Assume `stairs_mask` is a binary 2D numpy array representing the mask with the stairs labeled as 1.
# stairs_mask = (lbl_arr==2)
# H_img = lbl_arr.shape[0]  # Height of image

# # extract bottom pixels of the door
# door_bottom_pixels = extract_feature_pixels(door_mask,extract='bottom')
# print("Door bottom pixels:", door_bottom_pixels)

# # extract bottom pixels of the stairs
# stairs_bottom_pixels = extract_feature_pixels(stairs_mask,extract='bottom')
# print("Stairs bottom pixels:", stairs_bottom_pixels)

In [ ]:

def estimate_Z_for_points(building_outline, building_center, camera_position, image_points, focal_length_px, sensor_width_mm, image_width_px):
    """
    Estimate the distance (Z) to specific points on the building using building outline geometry and image coordinates.
    
    Parameters:
    - building_outline: Polygon representing the building outline.
    - building_center: Point representing the building center.
    - camera_position: Tuple (x, y, z) representing the camera's real-world position.
    - image_points: List of (x, y) tuples representing the points in image coordinates.
    - focal_length_px: Focal length in pixels.
    - sensor_width_mm: Sensor width in mm.
    - image_width_px: Image width in pixels.
    
    Returns:
    - List of estimated Z distances for each point.
    """
    # Calculate the field of view (FOV) of the camera in radians
    fov = 2 * math.atan((sensor_width_mm / 2) / (focal_length_px / (image_width_px / sensor_width_mm)))
    
    # Calculate distances from the camera to each point in the image
    Z_distances = []
    for point in image_points:
        # Calculate the angle of the point relative to the principal point (center of the image)
        x = point[0] - (image_width_px / 2)
        angle = (x / focal_length_px) * (fov / image_width_px)
        
        # Create a line from the camera position in the direction of the point
        direction_line = LineString([camera_position, (camera_position.x + math.cos(angle), camera_position.y + math.sin(angle))])
        
        # Calculate intersection with the building outline
        intersection = building_outline.intersection(direction_line)
        
        # If there is an intersection, calculate the distance
        if not intersection.is_empty:
            Z = camera_position.distance(intersection)
        else:
            # If no intersection, estimate Z as the distance to the building center
            Z = camera_position.distance(building_center)
        
        Z_distances.append(Z)
    
    return Z_distances


In [ ]:
# # Camera and image parameters
# focal_length_mm = 12.5  # Focal length in mm (use 16 or 12.5)
# sensor_width_mm = 8.8  # Sensor width in mm (for AVT GC2450 camera)
# image_width_px = 2448  # Image width in pixels
# image_height_px = 2050  # Image height in pixels

# # Convert focal length to pixels
# focal_length_px = focal_length_to_pixels(focal_length_mm, sensor_width_mm, image_width_px)
# print(f"Focal length in pixels: {focal_length_px:.2f}")

# # Coordinates of the points in the image (in pixels)
# image_points = [(1672, 1420), (1671, 1538)]  # Example points representing pixel coordinates

# # Load the shapefile using geopandas
# shapefile_path = "path/to/building_outline.shp"  # Update with the actual path to your shapefile
# building_center_path = "path/to/building_center.shp"  # Update with the actual path to your building center shapefile
# gdf = gpd.read_file(shapefile_path)
# center_gdf = gpd.read_file(building_center_path)

# # Assume the shapefile contains a single polygon representing the building outline
# building_outline = gdf.geometry.iloc[0]
# building_center = center_gdf.geometry.iloc[0]

# # Camera position in real-world coordinates (e.g., GPS coordinates)
# camera_position = Point(0, 0)  # Example camera position

# # Estimate Z distances for the image points
# Z_distances = estimate_Z_for_points(building_outline, building_center, camera_position, image_points, focal_length_px, sensor_width_mm, image_width_px)
# Z1 = Z_distances[0]
# Z2 = Z_distances[1]

# # Calculate the height difference
# y1 = image_points[0][1]
# y2 = image_points[1][1]
# height_difference = calculate_height_difference(y1, y2, Z1, Z2, focal_length_px, image_height_px)
# print(f"Estimated height difference: {height_difference:.2f} meters")

In [ ]:
H_img = lbl_arr.shape[0]  # Height of image
# Camera and image parameters
focal_length_mm = 8.5  # Focal length in mm
sensor_width_mm = 8.8  # Sensor width in mm (for AVT GC2450 camera)
image_width_px = 2448  # Image width in pixels
image_height_px = 2050  # Image height in pixels

# Convert focal length to pixels
focal_length_px = focal_length_to_pixels(focal_length_mm, sensor_width_mm, image_width_px)
print(f"Focal length in pixels: {focal_length_px:.2f}")

# Coordinates of the points in the image (in pixels)
y1 = 1561  # Example vertical pixel coordinate for point 1
y2 = 1462  # Example vertical pixel coordinate for point 2

# Real-world distance to the building (in meters)
Z1 = 17  # Example distance to the building
Z2 = 17.5

# Calculate the height difference
height_difference = calculate_height_difference_perspective(y1, y2, Z1, Z2, focal_length_px, image_height_px)
print(f"Estimated height difference: {height_difference:.2f} meters")


In [ ]:
# import sqlite3

# # Connect to your .sdb file
# db_path = r'C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\GA_data_documentation\RICS_DB\cbd_141112_11404.sdb'  # Replace with the path to your .sdb file
# conn = sqlite3.connect(db_path)

# # Create a cursor object to interact with the database
# cursor = conn.cursor()

# # Check the tables in the database (optional, to know which table to use)
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# tables = cursor.fetchall()
# print("Tables:", tables)  # This will list all the tables in the database

# # Query multiple example rows from a specific table
# table_name = 'L'  # Replace with the name of the table you want to check
# query = f"SELECT * FROM {table_name} LIMIT 5;"  # Select a few rows (e.g., 5 rows) from the table
# cursor.execute(query)

# # Fetch and print the rows
# rows = cursor.fetchall()  # Get all selected rows (up to 5)
# for i, row in enumerate(rows):
#     print(f"Row {i+1}:", row)

# cursor.execute(f"PRAGMA table_info({table_name});")
# columns = cursor.fetchall()
# print("Columns:", columns)
# # Close the connection when done
# conn.close()
